### Load Data and create Pandas Dataframe

In [1]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_48caab10af4d48c09f2c03f188ce40fc = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Ffe4iIx09CWFKUwymTlBGFe9O5UuObClsauSnUvp2TRL',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_48caab10af4d48c09f2c03f188ce40fc.get_object(Bucket='capstoneproject-donotdelete-pr-k93kpdp2m3wgr5',Key='Toronto.Postcodes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Postal Code,Borough,Neighbourhood,Unnamed: 3
0,M1A,Not assigned,Not assigned,NaN
1,M2A,Not assigned,Not assigned,NaN
2,M3A,North York,Parkwoods,NaN
3,M4A,North York,Victoria Village,NaN
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN


### Clean Data #1

In [2]:
# drop empty column
df.drop(columns='Unnamed: 3', inplace=True)

# drop rows where boroughs are 'Not assigned'
DropList = df[(df['Borough'] == 'Not assigned') ].index
df.drop(DropList , inplace=True)

df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
# checking to see if any of the remaining features contain a 'Not assigned' entry in the Neighbourhood field

Neigh = df[(df['Neighbourhood'] == 'Not assigned') ].index
Neigh.shape

(0,)

In [4]:
df.shape

(103, 3)

### Insert Lat and Long Dataframe

In [5]:

body = client_48caab10af4d48c09f2c03f188ce40fc.get_object(Bucket='capstoneproject-donotdelete-pr-k93kpdp2m3wgr5',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_LL= pd.read_csv(body)
df_LL.shape


(103, 3)

In [6]:
# Number rows for both data frames are equal. Will both Dataframes to ensure the data matches.

df.sort_values(by=['Postal Code'], inplace=True)
df.head()


,Postal Code,Borough,Neighbourhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae


In [7]:
# Sort LL Dataframe

df_LL.sort_values(by=['Postal Code'])
df_LL.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#Check last entries of Dataframe

df.tail()

,Postal Code,Borough,Neighbourhood
98,M9N,York,Weston
107,M9P,Etobicoke,Westmount
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
152,M9W,Etobicoke,"Northwest, West Humber - Clairville"


In [9]:
# Check last entries of LL Dataframe

df_LL.tail()

,Postal Code,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [10]:
# add lat and long columns to the dataframe

lat = df_LL["Latitude"]
df = df.join(lat)

long = df_LL["Longitude"]
df = df.join(long)

# reset index
df.reset_index(inplace= True)
df.drop(columns='index', inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.692657,-79.264848
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.778517,-79.346556
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.725900,-79.340923
3,M1G,Scarborough,Woburn,43.695344,-79.318389
4,M1H,Scarborough,Cedarbrae,43.712751,-79.390197
